# Purpose of this notebook

This notebook covers how to start the Neuroglancer viewer, load in some data, and customize the Neuroglancer window all in Python. It assumes you already have a local copy of the Neuroglancer layer directories for the raw data, raw atlas and raw cells. These are on bucket here:


- raw data layer: /jukebox/LightSheetData/lightserv_testing/neuroglancer/201904_ymaze_cfos/rawdata_an21/
- raw atlas layer: /jukebox/LightSheetData/lightserv_testing/neuroglancer/201904_ymaze_cfos/rawatlas_an21/
- raw cells layer: /jukebox/LightSheetData/lightserv_testing/neuroglancer/201904_ymaze_cfos/rawcells_an21_dilated/

All together than are about 8 GB so copy them to your local machine if possible. It will speed up the load into Neuroglancer enormously because you will not need to use the VPN. 

## A quick note about Neuroglancer
Neuroglancer loads in datasets in "layers". A layer can be of type "image" (like the raw data layer above) or type "segmentation" (like an atlas layer or cell layer). The naming is a little confusing because both layer types refer to volumes (3-d arrays). In Neuroglancer, you can overlay multiple layers or view them side-by-side, turn them on and off, and lots more. See the tutorial at [https://brainmaps.princeton.edu/2020/05/getting-started-with-neuroglancer/](https://brainmaps.princeton.edu/2020/05/getting-started-with-neuroglancer/) to learn more about what you can do with Neuroglancer.

# Setup
In order to run the code in this notebook, you will need an anaconda environment with python3 and containing some additional libraries. This environment "ng_ymaze" can be set up in the following way:
In terminal:
- conda create -n ng_ymaze python=3.7.4 -y
- conda activate ng_ymaze # (or source activate ng_mriatlas, depending on which version of conda you have)
- pip install cloud-volume
- **pip install neuroglancer==1.1.6** <br>
- pip install --user ipykernel # to enable using this conda environment as a jupyter kernel
- python -m ipykernel install --user --name=ng_ymaze

At this point, make sure to select this conda environment as the kernel when running this notebook (you might have to restart the notebook server, i.e. re-run jupyter notebook from the terminal)

## Host your layers on your local computer's network
This is a lot easier than it sounds. Open up a new terminal window on your computer and enter your ng_ymaze conda environment:
- conda activate ng_ymaze

Now enter a python interactive shell:
- python

Now execute the following lines in the python session:
```python
raw_layer_dir = "/path/to/wherever/you/copied/the/raw/layer" # this needs to be the full path
from cloudvolume import CloudVolume
vol = CloudVolume(f'file://{raw_layer_dir}')
vol.viewer(port=1337)
```

This should cause your session to hang -- that is expected. 

Now open up another terminal window and do the same thing but for the raw atlas, e.g.:
- conda activate ng_ymaze
- python 
Then, in python:
```python
rawatlas_layer_dir = "/path/to/wherever/you/copied/the/atlas/layer" # this needs to be the full path
from cloudvolume import CloudVolume
vol = CloudVolume(f'file://{rawatlas_layer_dir}')
vol.viewer(port=1338) # Note that this is 1338, versus 1337 above. This port needs to be different for each layer
```

Finally, do the same thing but for the raw cells:
- conda activate ng_ymaze
- python 

Then, in python:
```python
rawcells_layer_dir = "/path/to/wherever/you/copied/the/cells/layer" # this needs to be the full path
from cloudvolume import CloudVolume
vol = CloudVolume(f'file://{rawcells_layer_dir}')
vol.viewer(port=1339) # Note that this is 1339, versus 1337 and 1338 above. This port needs to be different for each layer
```

## Load your layers into Neuroglancer and view them

In [1]:
import neuroglancer 

In [8]:
# use the braincogs client
neuroglancer.set_static_content_source(url='https://nglancer.pni.princeton.edu')

viewer = neuroglancer.Viewer()
with viewer.txn() as s:
    s.layers['rawdata_an21'] = neuroglancer.ImageLayer(source='precomputed://http://localhost:1337'
    )
    s.layers['rawatlas_an21'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1338'
    )
    s.layers['rawcells_an21'] = neuroglancer.SegmentationLayer(source='precomputed://http://localhost:1339'
    )
# inverts the colormap. To do this in the actual viewer, just hit the "i" key
with viewer.txn() as s:
    s.layers['rawdata_an21'].layer.shader = """void main() {emitGrayscale(1.0-toNormalized(getDataValue())*15.0);} """

print(viewer)

http://127.0.0.1:34078/v/e0987ce39b83a6de75f6bfe5bdab662483699a8f/


When you run the cell above, it should generate a link for you. Click that link and you should be brought to Neuroglancer with your 3 layers loaded in.

The first thing you probably want to do.


In [9]:
with viewer.txn() as s:
    print(dir(s))

['__class__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '_cached_wrappers', '_get_wrapped', '_json_data', '_lock', '_readonly', '_set_wrapped', 'concurrentDownloads', 'concurrent_downloads', 'crossSectionBackgroundColor', 'cross_section_background_color', 'gpuMemoryLimit', 'gpu_memory_limit', 'interpolate', 'layers', 'layout', 'navigation', 'perspectiveOrientation', 'perspectiveViewBackgroundColor', 'perspectiveZoom', 'perspective_orientation', 'perspective_view_background_color', 'perspective_zoom', 'position', 'selectedLayer', 'selected_layer', 'showAxisLines', 'showDefaultAnnotations', 'showScaleBar', 'showSlices', 'show_axis_lines', 'show_default_annotations', 'show_scale_bar', 'show_slices', 'st

In [10]:
with viewer.txn() as s:
    print(s.perspective_oom)

None
